In [14]:
import time
import numpy as np

'''
This is one of the us_dist_sensor node rewritten such that the logic can be checked.
Refer to the test block for testing
'''



def publish_distance(tot_len, run_time, sig_len=5000, rand=True, distance_threshold=15, wait_time=5, rate=1):

    pos_sig = 0
    record_msg = "false"

    for i in range(run_time):
        distance = measure_distance(tot_len, sig_len=sig_len, rand=rand)

        if distance < distance_threshold:
            if pos_sig >= wait_time:
                record_msg = "true"  # Publish "true" to indicate recording
            elif pos_sig > 0 and record_msg == "true":
                pos_sig += 1
                record_msg = "true"
            else:
                pos_sig += 1
                record_msg = "false"  # Publish "false" to indicate not recording
        else:
            if pos_sig > 0 and record_msg == "true":
                pos_sig -= 1
                record_msg = "true"  # Publish "false" to indicate not recording
            else:
                pos_sig = 0
                record_msg = "false"  # Publish "false" to indicate not recording

        
        # print(pos_sig)
        time.sleep(rate)

        # Equivalent of publishing the record message
        print(record_msg)
    return

def measure_distance(tot_len, sig_len=100, rand=True):

    signal = trig_gen(tot_len, sig_len=sig_len, rand=rand)
    sig_now = 0
    i = 0

    start_time = time.perf_counter()
    stop_time = time.perf_counter()
    
    while sig_now == 0:
        start_time = time.perf_counter()
        i += 1
        sig_now = signal[i]
    while sig_now == 1:
        stop_time = time.perf_counter()
        i += 1
        sig_now = signal[i]
        
    elapsed_time = stop_time - start_time
    distance = (elapsed_time * 34300) / 2  # Speed of sound in cm/s
    # print(distance)
    return distance


# Mock distance sensor
def trig_gen(tot_len, sig_len=100, rand=True):

    if rand:
        f_half = int(tot_len)//2
        cutoff = np.random.randint(1, f_half-1)
    else:
        cutoff = (tot_len - sig_len)//2

    sig = np.zeros(tot_len)
    sig[cutoff:-cutoff] = 1
    return sig



In [18]:
'''
Test block
'''

#Randomised values distance sensor signals
# publish_distance(10000, 10, distance_threshold=40)

# Far away for 10 seconds
# Close for 10 seconds
# Far for 10 seconds
publish_distance(10000, 10, sig_len=8000, rand=False, distance_threshold=20)
publish_distance(10000, 10, sig_len=1000, rand=False, distance_threshold=20) #Note that with wait_time = 5, we wait 5 seconds for the object to be close before printing true
publish_distance(10000, 10, sig_len=8000, rand=False, distance_threshold=20)



false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
true
true
true
true
true
false
false
false
false
false
false
false
false
false
false
